In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import datetime
import os.path
import backtrader as bt
import pyfolio as pf
import pandas as pd

## Strategy

In [ ]:
# Create a Stratey
class TestStrategy(bt.Strategy):
    params = (
        ('maperiod', 15),
        ('printlog', False),
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function for this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # Add a MovingAverageSimple indicator
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.maperiod)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] > self.sma[0]:

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            if self.dataclose[0] < self.sma[0]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

    def stop(self):
        self.log('(MA Period %2d) Ending Value %.2f' %
                 (self.params.maperiod, self.broker.getvalue()), doprint=True)

## AcctValue & AcctStats Class

In [ ]:
class AcctValue(bt.Observer):
    alias = ('Value',)
    lines = ('value',)
 
    plotinfo = {"plot": True, "subplot": True}
 
    def next(self):
        self.lines.value[0] = self._owner.broker.getvalue()    # Get today's account value (cash + stocks)
        
class AcctStats(bt.Analyzer):
    """A simple analyzer that gets the gain in the value of the account; should be self-explanatory"""
 
    def __init__(self):
        self.start_val = self.strategy.broker.get_value()
        self.end_val = None
 
    def stop(self):
        self.end_val = self.strategy.broker.get_value()
 
    def get_analysis(self):
        return {"start": self.start_val, "end": self.end_val,
                "growth": self.end_val - self.start_val, "return": self.end_val / self.start_val}    

## Trade Analyzer and SQN print functions

In [ ]:
def printTradeAnalysis(analyzer):
    '''
    Function to print the Technical Analysis results in a nice format.
    
    Total trades still open
    Total closed trades
    Total trades won
    Total trades lost
    My Strike rate (which I will calculate from the data given)
    My best winning streak
    My worst losing streak
    My profit or loss.
    The SQN number of the strategy
    
    '''
    #Get the results we are interested in
    total_open = analyzer.total.open
    total_closed = analyzer.total.closed
    total_won = analyzer.won.total
    total_lost = analyzer.lost.total
    win_streak = analyzer.streak.won.longest
    lose_streak = analyzer.streak.lost.longest
    pnl_net = round(analyzer.pnl.net.total,2)
    strike_rate = round((total_won / total_closed) * 100,2)
    #Designate the rows
    h1 = ['Total Open', 'Total Closed', 'Total Won', 'Total Lost']
    h2 = ['Strike Rate','Win Streak', 'Losing Streak', 'PnL Net']
    r1 = [total_open, total_closed,total_won,total_lost]
    r2 = [strike_rate, '/', win_streak, lose_streak, pnl_net]
    #Check which set of headers is the longest.
    if len(h1) > len(h2):
        header_length = len(h1)
    else:
        header_length = len(h2)
    #Print the rows
    print_list = [h1,r1,h2,r2]
    row_format ="{:<15}" * (header_length + 1)
    print("Trade Analysis Results:")
    for row in print_list:
        print(row_format.format('',*row))

def printSQN(analyzer):
    sqn = round(analyzer.sqn,2)
    print('SQN: {}'.format(sqn))

## Strategy Plot

In [ ]:
class FixedSlippageAndCommisionScheme(bt.CommInfoBase):
    '''Use this for both slippage and commissions together'''
    # Using $2.37 commission(0.85)+Exchange(1.50)+Regulatory fee(2c) + 1 tick ($10) slippage per contract for CL.
    # source: https://www.interactivebrokers.com/en/index.php?f=1590&p=futures1
    params = (
        ('commission', 12.38),
        ('stocklike', True),
        ('commtype', bt.CommInfoBase.COMM_FIXED),
        )

    def _getcommission(self, size, price, pseudoexec):
        return self.p.commission

In [ ]:
# Initialize defaults, this code can be re-factored later to take these as CLI args or function/JSON config file args:
startingCash = 100000.0   # cerebro.broker.setcash
stake_Size = 1000  # cerebro.addsizer

# Create a cerebro entity
cerebro = bt.Cerebro(stdstats=False)  # https://www.backtrader.com/docu/observers-and-statistics/observers-and-statistics/

# Add a strategy
cerebro.addstrategy(TestStrategy)

# Load data
fname_symbol = 'CL'
folder_name = '5min'
suffix = '5min_20160103_20190405'

df = pd.read_parquet(os.path.join('../data/processed/{}/'.format(folder_name), '{}_{}.parquet'.format(fname_symbol, suffix)))
df = (df.resample('4h', label='left', base=18).agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'}))
df.columns = [col_name.lower() for col_name in df.columns]
df = df.dropna()

# Split the data into In-sample and Out-Of-Sample
OOSratio = 0.20
train_len = int(len(df)* (1 - OOSratio) )
dataIS = df.iloc[0:train_len]
dataOOS =  df.iloc[train_len:]

data = bt.feeds.PandasData(dataname = dataIS)

# Add the Data Feed to Cerebro
cerebro.adddata(data)   # << IN-SAMPLE data (for optimization).

# Set our desired cash start
cerebro.broker.setcash(startingCash)

cerebro.addsizer(bt.sizers.FixedSize, stake=stake_Size)

# Add Observers - since I set param stdstats=False when instantiating bt.Cerebro()
cerebro.addobserver(AcctValue)
cerebro.addobservermulti(bt.observers.BuySell)    # Plots up/down arrows

# Add Analyzers
cerebro.addanalyzer(AcctStats)
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="ta")
cerebro.addanalyzer(bt.analyzers.SQN, _name="sqn")

# We're trading futures, so we'll use the "commissions" to handle both slippage and commissions
slippage_and_comms = FixedSlippageAndCommisionScheme()
cerebro.broker.addcommissioninfo(slippage_and_comms)

# Alternatively, you can use:
# cerebro.broker.setcommission(commission=0.0)
# cerebro.broker.set_slippage_fixed(.00,  slip_open=True, slip_limit=True, slip_match=True, slip_out=True)


# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
print("Running in-sample optimization...")
cerebro.run()

# Print out the final result
print('\nFinal Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.plot(volume=False, iplot=True)

## In-sample Optimization

In [ ]:
# Create a cerebro entity
cerebro = bt.Cerebro()

maPeriodRange = range(10, 31)  

'''
maPeriodRange = []
_ = 3
maPeriodRange.append(_)
for i in range(11):
    _ *= 1.34
    maPeriodRange.append(int(round(_,0)))
'''    
    
print("Optimizing over MA Periods: ",maPeriodRange)

# Add a strategy
strats = cerebro.optstrategy(
    TestStrategy,
    maperiod=maPeriodRange)

# Add the Data Feed to Cerebro
cerebro.adddata(data)   # Still using the IN-SAMPLE data, for optimization..

# Set our desired cash start
cerebro.broker.setcash(startingCash)


# Add Observers (since I set param stdstats=False in bt.Cerebro() instantiation above)
cerebro.addobserver(AcctValue)
cerebro.addobservermulti(bt.observers.BuySell)    # Plots up/down arrows

# Add Analyzers
cerebro.addanalyzer(AcctStats)
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="ta")
cerebro.addanalyzer(bt.analyzers.SQN, _name="sqn")
cerebro.addanalyzer(bt.analyzers.DrawDown, _name="drawdown")
cerebro.addanalyzer(bt.analyzers.AnnualReturn, _name="annual_return")
cerebro.addanalyzer(bt.analyzers.SharpeRatio_A, timeframe=bt.TimeFrame.Days, riskfreerate=0, _name="sharpe_ratio")
cerebro.addanalyzer(bt.analyzers.VWR, timeframe=bt.TimeFrame.Days, tann=252, _name="VWR")
cerebro.addanalyzer(bt.analyzers.Calmar, timeframe=bt.TimeFrame.Days, _name="calmar")
# cerebro.addanalyzer(bt.analyzers.Calmar, timeframe=bt.TimeFrame.Minutes, compression=240, _name="calmar")  # Calmar ratio for 4-hour (240 min) timeframe.
cerebro.addanalyzer(bt.analyzers.PyFolio, timeframe=bt.TimeFrame.Days, compression=1)
cerebro.addanalyzer(bt.analyzers.Transactions,_name="transactions")
cerebro.addanalyzer(bt.analyzers.Returns,_name="returns")


# Add a FixedSize sizer according to the stake
cerebro.addsizer(bt.sizers.FixedSize, stake=stake_Size)

# We're trading futures, so we'll use the "commissions" to handle both slippage and commissions
slippage_and_comms = FixedSlippageAndCommisionScheme()
cerebro.broker.addcommissioninfo(slippage_and_comms)

# Run over everything
%time results = cerebro.run(maxcpus=1)

In [ ]:
# Code snippet - how to individually access each ma_period optimization result from the Analyzer:
# Results is a list of bt.cerebro.OptReturn objects, and each object is a single-item list itself, thus you need to reference the results object using [n][0], ie.
# the first index refers to a specific optimization run, eg. [0] refers to ma_period 10 run. The 2nd [0] must always be there, because as stated above, it is a list of OptReturn objects and each item in the list is a single-item list itself.

#print(results[6][0].params.maperiod)
#print(results[6][0].analyzers.calmar.get_analysis())

In [ ]:
# TESTING ONLY - Code snippet to neatly print the analyzer results:

'''
 RESOLVED (keep comment for future reference): Why was there a difference in the Sharpe Ratio's between the bt.analyzer.SharpeRatio and PyFolio's Sharpe ratio?
 I was using bt.analyzers.sharpe_ratio instead of analyzers.SharpeRatio_A for ANNUALIZED SR.
 It can also be due to: risk-free rate differences (PyFolio assumes 0%, backtrader uses 1% by default, need to set riskfreerate=0) 
 & timeframe (daily/monthly/yearly) makes a difference, even if it is annualized.
 ie. Sharpe ratio calculated on a higher timeframe inflates the sharpe ratio (eg. higher SR on Yearly timeframe compared to Daily.)
 https://community.backtrader.com/topic/680/sharpe-ratio-bt-vs-pyfolio-different-values
 https://community.backtrader.com/topic/1672/sharpe-ratio-wrong/3
'''
    
for strat in results:
    print('='*79)
    print(strat[0].__class__.__name__)
    print("MA Period: ",strat[0].params.maperiod)

    # Analyzers results
    #strat[0].analyzers.annualreturn.print()
    #strat[0].analyzers.drawdown.print()
    strat[0].analyzers.VWR.print()
    #strat[0].analyzers.sharpe_ratio.print()
    strat[0].analyzers.sqn.print()
    strat[0].analyzers.returns.print()

    # pyfolio
    pyfoliozer = strat[0].analyzers.getbyname('pyfolio')
    returns, positions, transactions, gross_lev = pyfoliozer.get_pf_items()

    # Perfomance
    pf.show_perf_stats(returns=returns)    

In [ ]:
for r in results:
    print(" "*25,"MA Period: ",r[0].params.maperiod)
    print('\n')
    printTradeAnalysis(r[0].analyzers.ta.get_analysis())
    printSQN(r[0].analyzers.sqn.get_analysis())
    print('='*79)
    print('\n')

In [ ]:
from IPython.display import display, HTML

# Get optimal combination

opt_res = pd.DataFrame({r[0].params.maperiod: r[0].analyzers.acctstats.get_analysis() for r in results}).T.loc[:, "return"].sort_values(ascending=False).index[0]
print("\nOptimum MA Period: {}".format(opt_res))

# Display stats of each MA Period backtest:
return_opt = pd.DataFrame({r[0].params.maperiod: r[0].analyzers.acctstats.get_analysis() for r in results}
                      ).T.loc[:, ['end', 'growth', 'return']]
display(return_opt)


## In-Sample equity curve & Sharpe

In [ ]:
import basictradestats #from https://community.backtrader.com/topic/670/it-s-here-a-beta-you-can-use-right-now-essential-trade-statistics-all-in-one-place/2
# also http://actuarialdatascience.com/backtrader_performance_report.html

# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(TestStrategy, maperiod=opt_res)

data = bt.feeds.PandasData(dataname = dataIS, plot=False)  # plot=False causes the chart to not be plotted, only the equity curve after adding the Value observer.

# Add the Data Feed to Cerebro
cerebro.adddata(data)

# Set our desired cash start
cerebro.broker.setcash(startingCash)
cerebro.addsizer(bt.sizers.FixedSize, stake=stake_Size)

# Add Sharpe-ratio analyzer:
cerebro.addanalyzer(bt.analyzers.SharpeRatio_A, timeframe=bt.TimeFrame.Days, riskfreerate=0, _name="sharpe_ratio")
cerebro.addanalyzer(bt.analyzers.PyFolio, timeframe=bt.TimeFrame.Days, compression=1)
cerebro.addanalyzer(bt.analyzers.TimeReturn, timeframe=bt.TimeFrame.Days, _name='dayreturn')
cerebro.addanalyzer(basictradestats.BasicTradeStats)

cerebro.addobserver(bt.observers.Value, plot=True)

# We're trading futures, so we'll use the "commissions" to handle both slippage and commissions
slippage_and_comms = FixedSlippageAndCommisionScheme()
cerebro.broker.addcommissioninfo(slippage_and_comms)

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
result = cerebro.run(stdstats=False)  # stdstats = False so it doesn't plot the Trades, Broker value, chart, etc.

cerebro.plot(volume=False, iplot=False)

daily_returns = result[0].analyzers.dayreturn.get_analysis()

# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())


result[0].analyzers.basictradestats.print()

# Print the Sharpe Ratio
print('Sharpe Ratio: {0:.2f}'.format(result[0].analyzers.sharpe_ratio.get_analysis()['sharperatio']))

# pyfolio
pyfoliozer = result[0].analyzers.getbyname('pyfolio')
returns, positions, transactions, gross_lev = pyfoliozer.get_pf_items()

# Perfomance
pf.show_perf_stats(returns=returns)    

## Out-of-Sample

In [ ]:
import basictradestats #from https://community.backtrader.com/topic/670/it-s-here-a-beta-you-can-use-right-now-essential-trade-statistics-all-in-one-place/2
# also http://actuarialdatascience.com/backtrader_performance_report.html

# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(TestStrategy, maperiod=opt_res)

data = bt.feeds.PandasData(dataname = dataOOS, plot=False)  # plot=False causes the chart to not be plotted, only the equity curve after adding the Value observer.

# Add the Data Feed to Cerebro
cerebro.adddata(data)

# Set our desired cash start
cerebro.broker.setcash(startingCash)
cerebro.addsizer(bt.sizers.FixedSize, stake=stake_Size)

# Add Sharpe-ratio analyzer:
cerebro.addanalyzer(bt.analyzers.SharpeRatio_A, timeframe=bt.TimeFrame.Days, riskfreerate=0, _name="sharpe_ratio")
cerebro.addanalyzer(bt.analyzers.PyFolio, timeframe=bt.TimeFrame.Days, compression=1)
cerebro.addanalyzer(bt.analyzers.TimeReturn, timeframe=bt.TimeFrame.Days, _name='dayreturn')
cerebro.addanalyzer(basictradestats.BasicTradeStats)

cerebro.addobserver(bt.observers.Value, plot=True)

# We're trading futures, so we'll use the "commissions" to handle both slippage and commissions
slippage_and_comms = FixedSlippageAndCommisionScheme()
cerebro.broker.addcommissioninfo(slippage_and_comms)

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
result = cerebro.run(stdstats=False)

daily_returns = result[0].analyzers.dayreturn.get_analysis()

# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.plot(volume=False, iplot=False)

result[0].analyzers.basictradestats.print()

# Print the Sharpe Ratio
print('Sharpe Ratio: {0:.2f}'.format(result[0].analyzers.sharpe_ratio.get_analysis()['sharperatio']))

# pyfolio
pyfoliozer = result[0].analyzers.getbyname('pyfolio')
returns, positions, transactions, gross_lev = pyfoliozer.get_pf_items()

# Perfomance
pf.show_perf_stats(returns=returns)    